In [1]:
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
cwd = os.chdir(r'C:\Users\rsurs\OneDrive\Documents\Zerodha\api_keys')
#print(f"current dir: {os.getcwd()}")
request_token = open('request_token.txt','r').read()
key_secret = open('credentials.txt','r').read().split()
access_token = open('access_token.txt','r').read()
api_key = key_secret[0]
api_secret = key_secret[1]
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [3]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [4]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [5]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [6]:
print(instrumentLookup(instrument_df,'RELIANCE'))

738561


In [7]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data


In [8]:
ohlc_df = fetchOHLC('ICICIBANK','5minute',5)

In [9]:
ohlc_df.head()

,open,high,low,close,volume
date,,,,,
2024-05-29 09:15:00+05:30,1115.05,1123.55,1115.05,1121.60,600076
2024-05-29 09:20:00+05:30,1121.40,1121.85,1118.30,1119.55,253766
2024-05-29 09:25:00+05:30,1119.85,1120.35,1117.75,1118.25,153792
2024-05-29 09:30:00+05:30,1117.85,1118.40,1117.00,1117.75,113086
2024-05-29 09:35:00+05:30,1118.00,1118.10,1115.55,1116.20,136577


In [10]:
ohlc_df.to_csv('ohlc.csv')

In [11]:
df = pd.read_csv('ohlc.csv')
df.set_index('date',inplace=True)
df.head()

,open,high,low,close,volume
date,,,,,
2024-05-29 09:15:00+05:30,1115.05,1123.55,1115.05,1121.60,600076
2024-05-29 09:20:00+05:30,1121.40,1121.85,1118.30,1119.55,253766
2024-05-29 09:25:00+05:30,1119.85,1120.35,1117.75,1118.25,153792
2024-05-29 09:30:00+05:30,1117.85,1118.40,1117.00,1117.75,113086
2024-05-29 09:35:00+05:30,1118.00,1118.10,1115.55,1116.20,136577


In [12]:
n=50

In [13]:
# df['sma'] = df['close'].rolling(n).mean()
# df

In [14]:
# df.dropna(inplace=True)

In [15]:
# df.head()

In [16]:
# multiplier = 2/(n + 1)

In [17]:
# df['ema'] = np.nan

In [18]:
# df

In [19]:
# for i in range(len(df)):
#     if i == 1:
#         df['ema'].iloc[i] = (df['close'].iloc[i] - df['sma'].iloc[i-1]) + df['sma'].iloc[i-1]
#     else:
#         df['ema'].iloc[i] = (df['close'].iloc[i] - df['ema'].iloc[i-1]) + df['ema'].iloc[i-1]

In [20]:
# df

In [21]:
# How to compute ema ?

# 1. Calculate the SMA

# (Period Values / Number of Periods)
# 2. Calculate the Multiplier

# (2 / (Number of Periods + 1) therefore (2 / (5+1) = 33.333%
# 3. Calculate the EMA
# For the first EMA, we use the SMA(previous day) instead of EMA(previous day).

# EMA = {Close - EMA(previous day)} x multiplier + EMA(previous day)


# How to compute macd ?

# Calculate the short-term EMA (Exponential Moving Average): This is often based on a 12-period EMA.
# Calculate the long-term EMA: This is often based on a 26-period EMA.
# Calculate the MACD Line: Subtract the long-term EMA from the short-term EMA.
# Calculate the Signal Line: Calculate a 9-period EMA of the MACD Line to create the Signal Line.


def compute_ema(df,n):
    df['sma'] = df['close'].rolling(n).mean()
    multiplier = 2/(n + 1)
    df.dropna(inplace=True)
    df['ema'] = np.nan
    ema_values = []
    for i in range(len(df)):
        if i == 1:
            df.loc[df.index[i], 'ema'] =  (df.loc[df.index[i],'close'] - df.loc[df.index[i - 1],'sma']) * multiplier + df.loc[df.index[i - 1], 'sma']
            #ema_values.append(df.loc[df.index[i], 'ema'])            
        else:
            df.loc[df.index[i], 'ema'] =  (df.loc[df.index[i],'close'] - df.loc[df.index[i - 1],'ema']) * multiplier + df.loc[df.index[i - 1], 'ema']
        ema_values.append(df.loc[df.index[i], 'ema'])
    return df[['ema']],ema_values

    

In [22]:
# ema_df,ema_vals = compute_ema(df,n=50)

In [23]:
# ema_vals

In [24]:
# ema_df

In [25]:
# df[['close','fast']] = compute_ema(df,n=12)
# df[['close','slow']] = compute_ema(df,n=26)
# df['macd'] = df['fast'] - df['slow']
# df[['close','signal']] = compute_ema(df[['close','macd']],n=9)
# df

In [26]:
df['fast'] = compute_ema(df, n=12)
df['slow'] = compute_ema(df, n=26)
df['macd'] = df['fast'] - df['slow']
df['signal'] = compute_ema(df[['macd']], n=9)


ValueError: Length of values (2) does not match length of index (289)

In [ ]:
df

,open,high,low,close,volume
date,,,,,
2024-05-29 09:15:00+05:30,1115.05,1123.55,1115.05,1121.60,600076
2024-05-29 09:20:00+05:30,1121.40,1121.85,1118.30,1119.55,253766
2024-05-29 09:25:00+05:30,1119.85,1120.35,1117.75,1118.25,153792
2024-05-29 09:30:00+05:30,1117.85,1118.40,1117.00,1117.75,113086
2024-05-29 09:35:00+05:30,1118.00,1118.10,1115.55,1116.20,136577
...,...,...,...,...,...
2024-06-03 15:05:00+05:30,1156.90,1159.10,1155.35,1158.65,928735
2024-06-03 15:10:00+05:30,1158.65,1160.00,1158.20,1159.40,667891
2024-06-03 15:15:00+05:30,1159.25,1164.40,1159.25,1163.70,921456
